# Preprocessing 

The data that we will be using to build the win probability model will be the wyscout open figshare data. 
The actual event data for an entire season is in a single large JSON file. We will use pandas to load in the data, process it into a format we want and build some basic features in this notebook. In the next notebook, we will continue to build the features and then finally train a model in the last notebook. 

In [2]:
import json
import os

import pandas as pd; pd.set_option("display.max_columns", None)
from pandas import json_normalize
import numpy as np
from tqdm import tqdm

In [3]:
WYSCOUT_DATA_FOLDER = "../wyscout_figshare_data" ## path where the wyscout data is extracted
PROCESSED_DATA_FOLDER = "../processed-data" ## where you want to save the processed csv files
EVENTS_FILE_NAME = os.path.join(WYSCOUT_DATA_FOLDER, "events/events_England.json") ## the league which you want to train the model on (Premier League here)
MATCHES_FILE_NAME = os.path.join(WYSCOUT_DATA_FOLDER, "matches/matches_England.json")

with open(EVENTS_FILE_NAME) as f:
    events = json.load(f)
df = json_normalize(events)
df['tags_list'] = df["tags"].apply(lambda x: [d['id'] for d in x])

with open(MATCHES_FILE_NAME) as f:
    matches = json.load(f)
matches_dict = {match['wyId']:match for match in matches} 

with open("../expected_threat.json") as f:
    xtd = np.array(json.load(f))

In [4]:
df.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,tags_list
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,[1801]
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,[1801]
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,[1801]
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,[1801]
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,[1801]


In [ ]:

def compute_xt_for_passes(pass_df, xtd, x_bins=12, y_bins=8):
    """Compute xt_value for successful passes (tag 1801)."""
    # explode positions into columns
    pos = pass_df["positions"].apply(
        lambda p: (p[0]["x"], p[0]["y"], p[1]["x"], p[1]["y"])
    )
    pass_df[["x1", "y1", "x2", "y2"]] = pd.DataFrame(pos.tolist(), index=pass_df.index)

    # bin indices (same as your code)
    pass_df["x1_idx"] = np.clip((pass_df["x1"].values // (100 / x_bins)).astype(np.int64), 0, x_bins - 1)
    pass_df["x2_idx"] = np.clip((pass_df["x2"].values // (100 / x_bins)).astype(np.int64), 0, x_bins - 1)
    pass_df["y1_idx"] = np.clip((pass_df["y1"].values // (100 / y_bins)).astype(np.int64), 0, y_bins - 1)
    pass_df["y2_idx"] = np.clip((pass_df["y2"].values // (100 / y_bins)).astype(np.int64), 0, y_bins - 1)

    pass_df["xt_value"] = (
        xtd[pass_df["y2_idx"].values, pass_df["x2_idx"].values]
        - xtd[pass_df["y1_idx"].values, pass_df["x1_idx"].values]
    )
    return pass_df


def process_match(match_df, match_md, xtd):
    """Return match_df with running goals/cards/players and xt_value filled."""
    # get team ids (same logic)
    home_team_id = int([k for k in match_md["teamsData"]
                        if match_md["teamsData"][k]["side"] == "home"][0])
    away_team_id = int([k for k in match_md["teamsData"]
                        if match_md["teamsData"][k]["side"] == "away"][0])

    match_df = match_df.copy()
    match_df["home_goals"] = 0
    match_df["away_goals"] = 0
    match_df["home_number_of_yellows"] = 0
    match_df["away_number_of_yellows"] = 0
    match_df["home_number_of_players"] = 11
    match_df["away_number_of_players"] = 11
    match_df["xt_value"] = np.nan

    # --- GOALS (Shots only) ---
    shots = match_df["eventName"].eq("Shot")
    tags = match_df["tags_list"]

    is_goal = tags.apply(lambda t: 101 in t)
    is_own_goal = tags.apply(lambda t: 102 in t)

    home_goal_event = shots & (
        (is_goal & match_df["teamId"].eq(home_team_id)) |
        (is_own_goal & match_df["teamId"].eq(away_team_id))
    )
    away_goal_event = shots & (
        (is_goal & match_df["teamId"].eq(away_team_id)) |
        (is_own_goal & match_df["teamId"].eq(home_team_id))
    )

    match_df["home_goals"] = home_goal_event.cumsum()
    match_df["away_goals"] = away_goal_event.cumsum()

    # --- YELLOW CARDS (tag 1702) ---
    yellow_event = tags.apply(lambda t: 1702 in t)
    home_yellow_event = yellow_event & match_df["teamId"].eq(home_team_id)
    away_yellow_event = yellow_event & match_df["teamId"].eq(away_team_id)

    match_df["home_number_of_yellows"] = home_yellow_event.cumsum()
    match_df["away_number_of_yellows"] = away_yellow_event.cumsum()

    # --- RED CARDS (tag 1701 or 1703) ---
    red_event = tags.apply(lambda t: (1701 in t) or (1703 in t))
    home_red_event = red_event & match_df["teamId"].eq(home_team_id)
    away_red_event = red_event & match_df["teamId"].eq(away_team_id)

    # each red reduces players from that point onward
    match_df["home_number_of_players"] = 11 - home_red_event.cumsum()
    match_df["away_number_of_players"] = 11 - away_red_event.cumsum()

    # --- PASS xT (successful passes tagged 1801) ---
    pass_mask = match_df["eventName"].eq("Pass") & tags.astype(str).str.contains("1801")
    pass_df = match_df.loc[pass_mask].copy()

    if not pass_df.empty:
        pass_df = compute_xt_for_passes(pass_df, xtd)
        match_df.loc[pass_df.index, "xt_value"] = pass_df["xt_value"]

    return match_df, home_team_id, away_team_id


match_ids = sorted(df["matchId"].unique())

for match_id in tqdm(match_ids):
    md = matches_dict[match_id]
    mdf = df.loc[df["matchId"].eq(match_id)]

    out_df, home_team_id, away_team_id = process_match(mdf, md, xtd)

    filename = os.path.join(PROCESSED_DATA_FOLDER, f"{match_id}_{home_team_id}_{away_team_id}.csv")
    out_df.to_csv(filename, index=False)

100%|██████████| 380/380 [00:12<00:00, 30.93it/s]
